## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 분류기는 여러 개 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측한다. (다수결 투표와 같은 방식)
간접 투표 분류기는 모든 분류기가 각 클래스의 확률을 예측할 수 있다는 전제 하에, 개별 분류기의 예측 확률을 평균내어 확률이 가장 높은 클래스를 선택한다.

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 감소시켜야 한다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [6]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [8]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)


####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [2]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [9]:
# model fitting
rf_clf=RandomForestClassifier(n_estimators=100,random_state=42)
extra_clf=ExtraTreesClassifier(n_estimators=100,random_state=42)
svm_clf=LinearSVC(random_state=42)
mlp_clf=MLPClassifier(random_state=42)

rf_clf.fit(X_train,y_train)
extra_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)
mlp_clf.fit(X_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [12]:
from sklearn.ensemble import VotingClassifier
voting_clf=VotingClassifier(
    estimators=[('rf', rf_clf),('extra',extra_clf),('svc',svm_clf),('mlp',mlp_clf)],
    voting='hard'
)

In [13]:
# model fitting
voting_clf.fit(X_train,y_train)



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('svc', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [14]:
# model test
from sklearn.metrics import accuracy_score

y_pred=voting_clf.predict(X_val)
print(accuracy_score(y_val,y_pred))


0.9686734693877551


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `set_params`를 활용해보세요

In [15]:
# 각 분류 모델 학습
rf_clf.fit(X_train,y_train)
extra_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)
mlp_clf.fit(X_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


MLPClassifier(random_state=42)

In [16]:
# 각 분류 모델의 성능 확인
rf_pred=rf_clf.predict(X_val)
extra_pred=extra_clf.predict(X_val)
svm_pred=svm_clf.predict(X_val)
mlp_pred=mlp_clf.predict(X_val)

print(accuracy_score(y_val,rf_pred))
print(accuracy_score(y_val,extra_pred))
print(accuracy_score(y_val,svm_pred))
print(accuracy_score(y_val,mlp_pred))

0.9673469387755103
0.9698979591836735
0.8288775510204082
0.9611224489795919


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. svm

In [17]:
# 가장 성능이 낮은 모델 제거
voting_clf2=VotingClassifier(
    estimators=[('rf', rf_clf),('extra',extra_clf),('mlp',mlp_clf)],
    voting='hard'
)


In [18]:
# model fitting
voting_clf2.fit(X_train,y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [19]:
# 모델 제거 후 성능 확인
y_pred2=voting_clf2.predict(X_val)
print(accuracy_score(y_val,y_pred2))


0.9716326530612245


### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [52]:
#rf_pred,extra_pred,svm_pred,mlp_pred 사용
X_blend=np.array([rf_pred,extra_pred,svm_pred,mlp_pred])
print(X_blend.shape)
X_blend=np.transpose(X_blend)
print(X_blend.shape)

(4, 9800)
(9800, 4)


####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [54]:
blend_clf=RandomForestClassifier(n_estimators=100,random_state=42)
blend_clf.fit(X_blend,y_val)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [53]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
rf_test_pred=rf_clf.predict(X_test)
extra_test_pred=extra_clf.predict(X_test)
svm_test_pred=svm_clf.predict(X_test)
mlp_test_pred=mlp_clf.predict(X_test)

In [55]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
blend_test_pred=np.array([rf_test_pred,extra_test_pred,svm_test_pred,mlp_test_pred])
blend_test_pred=np.transpose(blend_test_pred)

test_pred=blend_clf.predict(blend_test_pred)

In [56]:
# model test
print(accuracy_score(y_test,test_pred))

0.9658095238095238
